# Recombination & Retrival for Servoing

This notebook shows an example of how a suitable demonstration can be selected if several candidates are avaliable. The current selection is based on reprojection error.

Instructions:

    1. Record and Segment Demonstrations using `Record_and_Segment.ipynb`


## 1. Load Existing Demonstrations

In [ ]:
import os
import copy
import json
import logging
from glob import glob
from pathlib import Path

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

from flow_control.servoing.playback_env_servo import PlaybackEnvServo

root_dir = "/tmp/flow_experiments3"
task = "shape_sorting"
object_selected = "oval" 
task_variant = "rP"  # rotation plus (+-pi)

def get_configurations(root_dir=root_dir, num_episodes=20, prefix=""):
    os.makedirs(root_dir, exist_ok=True)
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    for seed in range(num_episodes):
        save_dir = save_dir_template + f"_{task_variant}"+f"_seed{seed:03d}"
        yield object_selected, seed, save_dir
            
demo_cfgs = get_configurations(prefix="demo")
recordings = []
for _, demo_seed, demo_dir in demo_cfgs:
    recordings.append(demo_dir)
    
print("Number of recordings:", len(recordings))
print("first", recordings[0])
print("last ", recordings[-1])

In [ ]:
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:]]

# If you get Warnings: Couldn't find servoing keyframes, you need to segment the demos
# This is shown in Record_and_Segment.ipynb 

In [ ]:
# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print("wp_name:", playbacks[demo_index][frame_index].get_info()["wp_name"])
    print("keep:", playbacks[demo_index].keep_dict.keys())
    fg_mask = playbacks[demo_index].get_fg_mask()
    if fg_mask is not None:
        print(f"percent fg @ {demo_index}, 0:", round(np.mean(fg_mask)*100,2))
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

# Filter Bad Demonstrations

In [ ]:
# Summarize the demonstration info
def filter_demo(pb):
    return pb[-1].data['rew'] > 0 and np.mean(pb.get_fg_mask()) > 0.005

demo_good = [filter_demo(pb) for pb in playbacks]

# print summary
fg_pixels = [np.mean(pb.get_fg_mask())*100 for pb in playbacks]
header = "ep\t len\t rew\t fg[%] bad"
print(header,"\n"+"-"*len(header.replace("\t"," "*5)))
for i, pb in enumerate(playbacks):
    print(i, "\t", len(pb), "\t", pb[-1].data["rew"],"\t", round(fg_pixels[i], 2),"\tx" if not demo_good[i] else '')    
print()

bad_pair_arr = np.zeros((len(playbacks), len(playbacks)), dtype=bool)
for idx in np.where(np.array(demo_good) == False)[0]:
    bad_pair_arr[:,idx] = True
    bad_pair_arr[idx,:] = True
bad_pair_arr += np.eye(len(playbacks), len(playbacks), dtype=bool)
#print(bad_pairs.astype(int))
good_pairs = list(zip(*np.where(bad_pair_arr==False)))

print(f"Good demos:   {np.mean(demo_good)*100:.1f} %\t", sum(demo_good),"/",len(demo_good) )
print(f"Viable pairs: {(1-bad_pair_arr.mean())*100:.1f} %\t",sum(bad_pair_arr.flatten()==0),"/", bad_pair_arr.size)



## Computer Similarity for Viable Pairs

Given the set of viable pairs, compute how similar the inital frames are.

In [ ]:
# Load Servoing Module
from flow_control.servoing.module import ServoingModule
control_config = dict(mode="pointcloud-abs-rotz", threshold=0.40)
servo_module = ServoingModule(demo_dir, control_config=control_config,
                              start_paused=False)

def similarity_from_reprojection(live_rgb, demo_rgb, demo_mask, return_images=False):
    # evaluate the similarity via flow reprojection error
    flow = servo_module.flow_module.step(demo_rgb, live_rgb)
    warped = servo_module.flow_module.warp_image(live_rgb / 255.0, flow)
    error = np.linalg.norm((warped - (demo_rgb / 255.0)), axis=2) * demo_mask
    error = error.sum() / demo_mask.sum()
    mean_flow = np.linalg.norm(flow[demo_mask],axis=1).mean()
    if return_images:
        return error, mean_flow, flow, warped
    return error, mean_flow

In [ ]:
# Plot similarity scores
%matplotlib notebook
fig, (ax, ax2, ax3) = plt.subplots(1, 3)
fig.suptitle("Similarity Scores")
ax.set_title("live")
ax2.set_title("demo")
ax3.set_title("warped")
ax.set_axis_off()
ax2.set_axis_off()
ax3.set_axis_off()
empty_image = np.zeros((256, 256), dtype=np.uint8)
image_h = ax.imshow(empty_image)
image_h2 = ax2.imshow(empty_image)
image_h3 = ax3.imshow(empty_image)

def update(live_index, demo_index):
    image_l = playbacks[live_index][0].cam.get_image()[0]
    image_h.set_data(image_l)
    ax.set_title(f"live @ {live_index}, 0")

    if not (demo_index, live_index) in good_pairs:
        image_h2.set_data(empty_image)
        image_h3.set_data(empty_image)
        ax2.set_title(f"demo")
        print("no valid demos.")
        return
    
    image_d = playbacks[demo_index][0].cam.get_image()[0]
    demo_mask =  playbacks[demo_index].get_fg_mask()
    error, mean_flow, flow, warped = similarity_from_reprojection(image_l, image_d, demo_mask,
                                                                  return_images=True)
    
    print(f"error {error:.2f} mean_flow {mean_flow:.2f}")    
    image_h2.set_data(image_d)
    image_h3.set_data(warped)
    ax2.set_title(f"demo @ {demo_index}, 0")
    fig.canvas.draw_idle()
    
slider_l = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=3,
                             layout=Layout(width='70%'))
slider_d = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=18,
                             layout=Layout(width='70%'))
interact(update, live_index=slider_l, demo_index=slider_d)
plt.tight_layout()
plt.show()

In [ ]:
sim_scores = np.ones(bad_pair_arr.shape)  # lower is better
mean_flows = np.zeros(bad_pair_arr.shape)

for live_i, demo_i in tqdm(good_pairs):
    live_rgb = playbacks[live_i][0].cam.get_image()[0]

    demo_rgb =  playbacks[demo_i][0].cam.get_image()[0]
    demo_mask =  playbacks[demo_i].get_fg_mask()
    error, mean_flow = similarity_from_reprojection(live_rgb, demo_rgb, demo_mask)
    assert error <= 1.0
    sim_scores[demo_i, live_i] = error
    mean_flows[demo_i, live_i] = mean_flow

### Normalize Errors
Rescale both reprojection error and mean flow to the range 0,1.
Combine these two to obtain normalized erros

In [ ]:
from sklearn.preprocessing import minmax_scale

sim_l = sim_scores[demo_good]
mean_flows_l = mean_flows[demo_good]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel("reprojection")
ax.set_ylabel("mean flow")
ax.scatter(minmax_scale(sim_l), minmax_scale(mean_flows_l))
plt.show()

sim_scores_norm = np.ones(sim_scores.shape)
w = .5
sim_scores_norm[demo_good] = np.mean((1*minmax_scale(sim_l), w*minmax_scale(mean_flows_l)),axis=0)/(1+w)

## 2. Pick Best Demonstrations

Given a list of all possible pairs, find the best demonstrations.
Also plot demonstrations by order of goodness.

In [ ]:
good_episode = np.any(demo_good, axis=0)
print("live episode", np.arange(len(recordings))[good_episode])
print("demo episode", np.argmin(sim_scores_norm, axis=0)[good_episode])

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2)
fig.suptitle("Ranked Similarity Scores")
ax.set_title("live")
ax2.set_title("demo")
ax.set_axis_off()
ax2.set_axis_off()
empty_image = np.zeros((256, 256), dtype=np.uint8)
image_h = ax.imshow(empty_image)
image_h2 = ax2.imshow(empty_image)

def update(live_index, demo_rank):
    image_l = playbacks[live_index][0].cam.get_image()[0]
    image_h.set_data(image_l)
    # chose d'th best demo
    demo_i = np.argsort(sim_scores_norm[:, live_index])[demo_rank]
    ax.set_title(f"live @ {live_index}, 0")
    ax2.set_title(f"rank {demo_rank} demo @ {demo_i}, 0")
    
    if not (demo_i, live_index) in good_pairs:
        image_h2.set_data(empty_image)
        print("no good demo.")
        return
    print(f"{demo_rank} th best demo is {demo_i}   rpj. err {sim_scores[demo_i, live_index]:.2f}   dist {mean_flows[demo_i, live_index]:.2f}")
    image_d = playbacks[demo_i][0].cam.get_image()[0]
    image_h2.set_data(image_d)
    fig.canvas.draw_idle()
    
slider_l = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=3,
                             layout=Layout(width='70%'))
slider_d = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=0,
                             layout=Layout(width='70%'))
interact(update, live_index=slider_l, demo_rank=slider_d)
plt.tight_layout()
plt.show()

## Run Servoing

For a live run, we know what the best demonstration is, evaluate this.

In [ ]:
import shutil
from math import pi
from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.servoing.module import ServoingModule
from flow_control.runner import evaluate_control

renderer = "debug"
control_config = dict(mode="pointcloud-abs-rotz", threshold=0.35)

def get_save_dir(root_dir, live_seed, demo_seed):
    prefix = "run"
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    save_dir = save_dir_template + f"_{task_variant}_seed{live_seed:03d}_demo{demo_seed:03d}" 
    return save_dir    

def eval_cmb(live_seed, demo_seed, exist_ok=False):
    save_dir = get_save_dir(root_dir, live_seed, demo_seed)
    if Path(save_dir).is_dir():
        if exist_ok:
            shutil.rmtree(save_dir)
        else:
            raise ValueError
    print("saving to:", save_dir)
    demo_dir = recordings[demo_seed]
    servo_module = ServoingModule(demo_dir, control_config=control_config,
                                  start_paused=False, plot=False, plot_save_dir=None)
    
    env = RobotSimEnv(task='shape_sorting', renderer=renderer, act_type='continuous',
                      initial_pose='close', max_steps=500, control='absolute-full',
                      img_size=(256, 256), param_randomize=("geom",), seed=int(live_seed),
                      task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]))
    
    _, reward, _, info = evaluate_control(env, servo_module, max_steps=130, save_dir=save_dir)
    return reward

# Run servoing for best combination.
def best_pairs():
    for live_i in np.arange(len(recordings)):
        demo_i = np.argmin(sim_scores_norm[:, live_i])
        if bad_pair_arr[demo_i, live_i]:
            continue
        yield live_i, demo_i
        
#live_seed, demo_seed = 3, 19
#reward = eval_cmb(live_seed, demo_seed, exist_ok=True)
#print(f"live_i {live_seed} demo_seed {demo_seed} -> reward {reward}")

In [ ]:
# set reward_arr to -1 to indicate that combination was not run
servo_rewards = np.zeros(bad_pair_arr.shape, dtype=int) - 1
for live_i, demo_i in best_pairs():
    print(f"eval pair: live={live_i}, demo={demo_i}")
    reward = eval_cmb(live_i, demo_i, exist_ok=True)
    servo_rewards[demo_i, live_i] = reward

In [ ]:
servo_rewards_m = np.ma.masked_array(servo_rewards, mask=servo_rewards == -1)
trials_per_live = servo_rewards_m.count(axis=0)
assert np.all(trials_per_live <= 1)
assert np.all(trials_per_live >= 0)
run_succ = np.sum(servo_rewards_m > 0)
run_total = trials_per_live.sum()
print(f"success rate: {servo_rewards_m.mean()*100:.1f}%\t{run_succ} / {run_total}\n")

demo_f, live_f = np.where(servo_rewards_m == 0)
demo_f = demo_f[np.argsort(live_f)]
live_f = sorted(live_f)
print("run fail:", " ".join([f"{l}w{d}" for l, 
                             d in zip(live_f,demo_f)]))

In [ ]:
#print("mean performance", tmp.mean())
#print("mean performance (masked): ", np.ma.masked_array(tmp, mask=demo_bad).mean().round(3))

reward_arr_m = np.ma.masked_array(servo_rewards, mask=bad_pair_arr)
fig, ax = plt.subplots()
fig.suptitle("Success for Demo/Live")
ax.set_xlabel("live episode index")
ax.set_ylabel("demo episode index")
ax.imshow(reward_arr_m)
plt.show()

# Load Recorded Servoing Runs

We have run servoing, now we load the recorded runs.

Also load the segmentation of the demonstration into parts, as was done by `Record_and_Segemnt.ipynb`

Don't worry about warnings about the missing `servo_keep.json` files, these are not neeeded.

In [ ]:
# Load Parts
demo_parts_fn = os.path.join(root_dir, "demo_parts_manual3.json")
with open(demo_parts_fn) as f_obj:
    demo_parts = json.load(f_obj)

def get_parts(demo_i, demo_frame):
    for part in demo_parts[str(demo_i)]:
        if demo_frame >= part["start"] and demo_frame <= part["end"]:
            return part["name"]
# Load runs
servo_run_fns = []
for live_i, demo_i in best_pairs():
    save_dir = get_save_dir(root_dir, live_i, demo_i)
    servo_run_fns.append((live_i,save_dir))
    
servo_playbacks = dict([(live_i,PlaybackEnvServo(run_fn)) for live_i, run_fn in servo_run_fns])
servo_demos = dict(best_pairs())

def get_demo_frames(run_index):
    demo_frames = []
    for frame_index in range(len(servo_playbacks[run_index])):
        demo_frame = servo_playbacks[run_index][frame_index].get_info()["demo_index"]
        demo_frames.append(demo_frame)
    return demo_frames

servo_max_len = max([len(spb) for spb in servo_playbacks.values()])
demo_frames = dict([(r_i, get_demo_frames(r_i)) for r_i in servo_playbacks])


In [ ]:
# Plot the servoing runs
%matplotlib notebook
import matplotlib.gridspec as gridspec

fig, ax = plt.subplots(figsize=(8, 6))
fig.suptitle("Servoing Frames")
g_s = gridspec.GridSpec(2, 2, figure=fig)
#g_s.update(wspace=0.001, hspace=.001)  # set the spacing between axes.
ax1 = fig.add_subplot(g_s[0, 0])
ax2 = fig.add_subplot(g_s[0, 1])
ax_p = fig.add_subplot(g_s[1, :])
ax.set_axis_off()
ax1.set_axis_off()
ax2.set_axis_off()
image_h = ax1.imshow(servo_playbacks[0].cam.get_image()[0])
image_h2 = ax2.imshow(playbacks[servo_demos[0]].cam.get_image()[0])

def plot_line(ax, data, frame_index=None):
    ax.clear()
    ax.plot(data,'.-')
    if frame_index is not None:
        ax.vlines([frame_index], 0, 10, color='k')

def update(run_index, frame_index):
    if run_index not in servo_playbacks:
        image_h.set_data(empty_image)
        image_h2.set_data(empty_image)
        ax_p.clear()
        return
    
    frame_index = min(frame_index, len(servo_playbacks[run_index])-1)
    image = servo_playbacks[run_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    ax1.set_title(f"run ({run_index}, {frame_index})")
    
    demo_i = servo_demos[run_index]
    demo_frame = servo_playbacks[run_index][frame_index].get_info()["demo_index"]
    image_d = playbacks[demo_i][demo_frame].cam.get_image()[0]
    image_h2.set_data(image_d)
    ax2.set_title(f"demo ({demo_i}, {demo_frame})")
    plot_line(ax_p, get_demo_frames(run_index), frame_index)
    ax_p.set_xlabel("run frame")
    ax_p.set_ylabel("demo frame")
    
    fig.canvas.draw_idle()
    
    print(get_parts(demo_i, demo_frame), demo_frame, demo_parts[str(demo_i)])
    print(servo_playbacks[run_index].get_info().keys())
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=11,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=servo_max_len-1, step=1, value=0,
                             layout=Layout(width='70%'))
    
interact(update, run_index=slider_w, frame_index=slider_i)
plt.tight_layout()
plt.show()

# Live Demo Switching

In live demo selection the aim is to find a more suitable demonstration - during the servoing run.

For this we first do some offline experiments based on the servoing runs that we have recorded.

To do this we consider the demonstration parts. Now we want to find sites where we can switch between demonstrations. This means that we look at demonstration parts ($P_i$) and find when servoing has converged with these.

In [ ]:
def get_demo_part_changes(demo_i):
    parts = demo_parts[str(demo_i)]
    changes = [part["start"] for part in parts if part["start"] != 0]
    return changes

# gives the frame number when a run has converged with the last frame of the
# first demonstration part.
end_part1 = {}

print("r d \t d.chg.  r.chg\t demo wp name.")
for run_index in servo_playbacks:
    demo_i = servo_demos[run_index]
    demo_frames = get_demo_frames(run_index)
    demo_part_changes = get_demo_part_changes(demo_i)
    wp_name = playbacks[demo_i][demo_part_changes[0]].get_info()["wp_name"]
    
    pcs = []
    for pc in demo_part_changes:
        try:
            # get the last occurance of pc in demo_frames
            pcs.append(len(demo_frames) - demo_frames[::-1].index(pc) - 1)
            # get the first occurance of pc in demo_frames
            #pcs.append(demo_frames.index(pc))
        except ValueError:
            pass
    print(run_index, demo_i,"\t", demo_part_changes, pcs, "\t", wp_name)
    if len(pcs):
        end_part1[run_index] = pcs[0]
        
#r. d., d. chng, run chng, demo wp_name

In [ ]:
# Plot the demonstrations
%matplotlib notebook
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(8, 6))
fig.suptitle("End-of-Part Demo Options")
ax1.set_axis_off()
ax2.set_axis_off()

image_h = ax1.imshow(playbacks[0].cam.get_image()[0])
image_h2 = ax2.imshow(empty_image)

def update(run_index, demo_index):
    if run_index in end_part1:
        frame_index = end_part1[run_index]
        ax1.set_title(f"run ({run_index}, {frame_index})")
        image = servo_playbacks[run_index][frame_index].cam.get_image()[0]
        image_h.set_data(image)
    else:
        ax1.set_title(f"run")
        image_h.set_data(empty_image)
        return
    print("run demo was:", servo_demos[run_index])
    df_i = get_demo_part_changes(demo_index)[0]
    ax2.set_title(f"demo ({demo_index}, {df_i})")
    image_d = playbacks[demo_index][df_i].cam.get_image()[0]
    image_h2.set_data(image_d)
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, run_index=slider_w, demo_index=slider_i)
plt.show()

In [ ]:
sim_scores = np.ones(bad_pair_arr.shape)  # lower is better
mean_flows = np.zeros(bad_pair_arr.shape)

for live_i, demo_i in tqdm(good_pairs):
    if live_i not in end_part1:
        continue
    frame_index = end_part1[live_i]
    live_rgb = servo_playbacks[live_i][frame_index].cam.get_image()[0]
    
    demo_f = get_demo_part_changes(demo_i)[0]
    demo_rgb =  playbacks[demo_i][demo_f].cam.get_image()[0]
    demo_mask =  playbacks[demo_i].get_fg_mask(demo_f)
    error, mean_flow = similarity_from_reprojection(live_rgb, demo_rgb, demo_mask)
    assert error <= 1.0
    sim_scores[demo_i, live_i] = error
    mean_flows[demo_i, live_i] = mean_flow

In [ ]:
good_episode = np.any(demo_good, axis=0)
print("live episode", np.arange(len(recordings))[good_episode])
print("demo episode", np.argmin(sim_scores, axis=0)[good_episode])

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2)
fig.suptitle("Demo Part - Ranked Similarity Scores")
ax.set_title("live")
ax2.set_title("demo")
ax.set_axis_off()
ax2.set_axis_off()
empty_image = np.zeros((256, 256), dtype=np.uint8)
image_h = ax.imshow(empty_image)
image_h2 = ax2.imshow(empty_image)

def update(run_index, demo_rank):
    print("run demo was:", servo_demos[run_index])
    if run_index in end_part1:
        frame_index = end_part1[run_index]
        ax1.set_title(f"run ({run_index}, {frame_index})")
        image = servo_playbacks[run_index][frame_index].cam.get_image()[0]
        image_h.set_data(image)
    else:
        image_h.set_data(empty_image)
        image_h2.set_data(empty_image)
        return
    # chose d'th best demo
    demo_i = np.argsort(sim_scores[:, run_index])[demo_rank]
    demo_f = get_demo_part_changes(demo_i)[0]

    ax.set_title(f"live @ {run_index}, {frame_index}")
    ax2.set_title(f"rank {demo_rank} demo @ {demo_i}, {demo_f}")
    
    if not (demo_i, run_index) in good_pairs:
        image_h2.set_data(empty_image)
        print("no good demo.")
        return
    print(f"{demo_rank} th best demo is {demo_i}   rpj. err {sim_scores[demo_i, run_index]:.2f}   dist {mean_flows[demo_i, run_index]:.2f}")
    image_d = playbacks[demo_i][demo_f].cam.get_image()[0]
    image_h2.set_data(image_d)
    fig.canvas.draw_idle()
    
slider_l = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=3,
                             layout=Layout(width='70%'))
slider_d = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=0,
                             layout=Layout(width='70%'))
interact(update, run_index=slider_l, demo_rank=slider_d)
plt.tight_layout()
plt.show()

In [ ]:
live_seed, demo_seed = 6, 7
reward = eval_cmb(live_seed, demo_seed, exist_ok=True)
print(f"live_i {live_seed} demo_seed {demo_seed} -> reward {reward}")

# Live Selection Run

Run the `Demo_Selection_Live.ipynb`